In [ ]:
!pip install transformers sentence-transformers scikit-learn tqdm

In [1]:
import pandas as pd
import json
import numpy as np
import tqdm
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
h=['year','id','title','journal_name','col5']
df1=pd.read_csv('/content/drive/MyDrive/casereports/defaultCR_selection.csv',on_bad_lines='skip',names=h)
df1.head()

,year,id,title,journal_name,col5
0,2024,PMC11197899,Bilateral Optic Neuroretinitis: Uncommon Compl...,Neuroophthalmology,"""Case Reports;Journal Article;"""
1,2024,PMC11095573,Bilateral Optic Neuropathy Associated with Acu...,Neuroophthalmology,"""Case Reports;"""
2,2024,PMC10899702,Telemedicine in patients with obsessive-compul...,Front Hum Neurosci,"""Case Reports;Journal Article;"""
3,2024,PMC11073879,Compulsive Repetitive Flexion With Breath-Hold...,Neurol Clin Pract,"""Case Reports;"""
4,2024,PMC10864136,Spontaneous Regression of Glioma-Mimicking Bra...,Brain Tumor Res Treat,"""Case Reports;Journal Article;"""


In [ ]:
h=['year','id','title','journal_name','col5']
df2=pd.read_csv('/content/drive/MyDrive/casereports/relatedCR_selection.csv',on_bad_lines='skip',names=h)
df2.head()

,year,id,title,journal_name,col5
0,2024,PMC10806872,Cytotoxic lesions of the corpus callosum (CLOC...,BMJ Neurol Open,"""Journal Article;"""
1,2024,PMC11192191,Childhood Cerebral Adrenoleukodystrophy: Case ...,Degener Neurol Neuromuscul Dis,"""Case Reports;Journal Article;"""
2,2024,PMC10968772,Effects of Cervical Spinal Manipulation on Sac...,Brain Sci,"""Case Reports;Journal Article;"""
3,2024,PMC11295153,A case of frontal lobe seizures with 'dancing-...,Eur J Neurol,"""Case Reports;Journal Article;"""
4,2024,PMC11022958,Cerebellar ependymal cyst: a case report.,Front Neurosci,"""Case Reports;Journal Article;"""


In [ ]:
df=pd.concat([df1,df2],ignore_index=True)
df.shape

(9343, 5)

In [ ]:
df.to_csv('/content/drive/MyDrive/casereports/titles.csv',index=False)

In [2]:
df=pd.read_csv('/content/drive/MyDrive/casereports/titles.csv',on_bad_lines='skip')
df.head()

,year,id,title,journal_name,col5
0,2024,PMC11197899,Bilateral Optic Neuroretinitis: Uncommon Compl...,Neuroophthalmology,"""Case Reports;Journal Article;"""
1,2024,PMC11095573,Bilateral Optic Neuropathy Associated with Acu...,Neuroophthalmology,"""Case Reports;"""
2,2024,PMC10899702,Telemedicine in patients with obsessive-compul...,Front Hum Neurosci,"""Case Reports;Journal Article;"""
3,2024,PMC11073879,Compulsive Repetitive Flexion With Breath-Hold...,Neurol Clin Pract,"""Case Reports;"""
4,2024,PMC10864136,Spontaneous Regression of Glioma-Mimicking Bra...,Brain Tumor Res Treat,"""Case Reports;Journal Article;"""


In [3]:
# Load domain keywords
with open("/content/drive/MyDrive/casereports/combined_domain_keywords.json") as f:
    domain_terms = json.load(f)

df['title_clean'] = df['title'].astype(str).str.lower().str.strip()

In [ ]:
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

# Load the model
model = SentenceTransformer("allenai/specter")

# Your titles
titles = df['title_clean'].tolist()

# Function to embed in batches
def embed_in_batches(texts, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        batch_embeddings = model.encode(batch, show_progress_bar=False)
        embeddings.extend(batch_embeddings)
    return np.array(embeddings)

# Run embedding
title_embeddings = embed_in_batches(titles, batch_size=32)

# Save to .npy
np.save("/content/drive/MyDrive/casereports/title_embeddings.npy", title_embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/321 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]


100%|██████████| 292/292 [00:24<00:00, 11.98it/s]


In [ ]:
import json
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from tqdm import tqdm

def refine_domain_terms(domain_terms,
                        overlap_threshold=7,
                        idf_threshold=2.0,
                        clustering=False,
                        max_clusters=5,
                        max_terms_per_domain=100,
                        embedding_model_name="allenai/specter"):
    """
    Cleans and refines a dictionary of domain-specific keywords.
    Removes overlapping terms, filters common (low IDF) terms,
    and optionally clusters to keep only representative terms.

    Parameters:
    - domain_terms: dict mapping domain name → list of terms
    - overlap_threshold: int, max domains a term can belong to
    - idf_threshold: float, min IDF score to keep a term
    - clustering: bool, whether to cluster terms per domain
    - max_clusters: int, number of clusters for KMeans
    - max_terms_per_domain: int, cap for final keywords per domain
    - embedding_model_name: huggingface model name (SPECTER by default)

    Returns:
    - refined_terms: dict of domain → list of cleaned, distinct terms
    """

    print("🔄 Step 1: Remove overlapping terms...")
    # Build term→domain map
    term_domain_map = defaultdict(set)
    for domain, terms in domain_terms.items():
        for term in terms:
            term_domain_map[term].add(domain)

    # Remove terms that appear in too many domains
    non_overlapping_terms = defaultdict(list)
    for domain, terms in domain_terms.items():
        for term in terms:
            if len(term_domain_map[term]) <= overlap_threshold:
                non_overlapping_terms[domain].append(term)

    print("🔍 Step 2: TF-IDF filtering...")
    # Convert domain-wise term lists to corpus
    domain_texts = [" ".join(terms) for terms in non_overlapping_terms.values()]
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(domain_texts)
    idf_scores = dict(zip(vectorizer.get_feature_names_out(), vectorizer.idf_))

    filtered_terms = defaultdict(list)
    for domain, terms in non_overlapping_terms.items():
        for term in terms:
            if idf_scores.get(term, 0) >= idf_threshold:
                filtered_terms[domain].append(term)

    if clustering:
        print("🧠 Step 3: Clustering to select representative terms...")
        model = SentenceTransformer(embedding_model_name)
        clustered_terms = {}

        for domain, terms in tqdm(filtered_terms.items()):
            terms = list(set(terms))[:2000]  # embed only top-N to save time/mem
            embeddings = model.encode(terms, convert_to_numpy=True, show_progress_bar=False)

            n_clusters = min(max_clusters, len(terms)) if len(terms) > 5 else len(terms)
            if n_clusters <= 1:
                clustered_terms[domain] = terms[:max_terms_per_domain]
                continue

            kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(embeddings)
            closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
            selected_terms = [terms[i] for i in closest]

            clustered_terms[domain] = selected_terms[:max_terms_per_domain]
        return clustered_terms

    print("✅ Skipping clustering. Returning filtered terms.")
    return filtered_terms


In [ ]:
# Load your original combined terms
with open("/content/drive/MyDrive/casereports/combined_domain_keywords.json") as f:
    domain_terms = json.load(f)

refined_terms = refine_domain_terms(domain_terms)

# Save the refined result
with open("/content/drive/MyDrive/casereports/refined_domain_keywords.json", "w") as f:
    json.dump(refined_terms, f, indent=2)

print("✅ Saved cleaned, deduplicated domain terms.")


🔄 Step 1: Remove overlapping terms...
🔍 Step 2: TF-IDF filtering...
✅ Skipping clustering. Returning filtered terms.
✅ Saved cleaned, deduplicated domain terms.


In [ ]:
with open("/content/drive/MyDrive/casereports/refined_domain_keywords.json") as f:
    refined_terms = json.load(f)

In [ ]:
for domain, terms in refined_terms.items():
    print(f"\nDomain: {domain} ({len(terms)} terms)")
    print(", ".join(terms[:10]))  # Show only first 10



Domain: gene (130602 terms)
0610007f13rik, 0610007i11rik, 0610008b10rik, 0610008c08rik, 0610008f14rik, 0610009m10rik, 0610010c24rik, 0610010i06rik, 0610010i13rik, 0610010m09rik

Domain: taxonomy (74011 terms)
44ahjd, aardvark, aardvarks, aardwolf, aav2, abalone, abalones, abele, abies, abiotrophia

Domain: brain_atlas (5415 terms)
10cb, 10dm, 10m, 10mc, 10mr, 10n, 10o, 10p, 10pp, 10v

Domain: anatomy (112945 terms)
10n, 12n, 15539390, 17093119, 17611285, 19496174, 3nf, 4nf, a10a, a42

Domain: molecule (40962 terms)
0000, 0000548579903, 0113428913, 10000347, 10001604, 10003985, 10005243, 10005290, 10008747, 10009590

Domain: disease (66091 terms)
aaas, aamr, aarrs, aat3, aat5, abcds, abetalipoproteinemia, abortion, abscess, acanthocephaliasis

Domain: metadata (24150 terms)
10q112, 10q12, 10q21, 10q213, 10q22, 10q23, 10q2331, 10q24, 10q252, 10q2612

Domain: protein (93418 terms)
11b6, 12e7, 15e11, 15e12, 16g2, 17betahsd11, 17betahsdxi, 17bhsd11, 1a13b, 1c7

Domain: phenotype (4747 term

In [ ]:
import json
import torch
import numpy as np
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
import os

In [ ]:
with open("/content/drive/MyDrive/casereports/refined_domain_keywords.json") as f:
    domain_terms = json.load(f)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = "allenai/specter"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to(device)

def embed_texts(texts, batch_size=16):
    all_embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i + batch_size]
        encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            model_output = model(**encoded_input)
        # Use [CLS] token as sentence embedding
        embeddings = model_output.last_hidden_state[:, 0, :].cpu().numpy()
        all_embeddings.extend(embeddings)
    return np.array(all_embeddings)


In [ ]:
domain_embeddings = {}

for domain, keywords in tqdm(domain_terms.items(), desc="Embedding domain keywords"):
    if not keywords:
        continue
    embeddings = embed_texts(keywords, batch_size=16)
    domain_embeddings[domain] = embeddings  # Store full array per domain


  0%|          | 0/8163 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.

 23%|██▎       | 952/4131 [00:10<00:41, 77.18it/s]

In [ ]:
import pickle
with open("/content/drive/MyDrive/casereports/domain_full_embeddings.pkl", "wb") as f:
    pickle.dump(domain_embeddings, f)

In [4]:
title_embeddings=np.load('/content/drive/MyDrive/casereports/title_embeddings.npy')

In [5]:
import pickle

path = "/content/drive/MyDrive/casereports/domain_full_embeddings.pkl"

with open(path, "rb") as f:
    domain_embeddings = pickle.load(f)

# Check keys (domains)
print("Loaded domains:", list(domain_embeddings.keys()))


Loaded domains: ['gene', 'taxonomy', 'brain_atlas', 'anatomy', 'molecule', 'disease', 'metadata', 'protein', 'phenotype', 'cognitive']


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Threshold for assigning a domain
THRESHOLD = 0.45

# Function to classify each title
def classify_title(embedding, domain_embeddings, threshold=THRESHOLD):
    predicted_domains = []

    for domain, keyword_embeds in domain_embeddings.items():
        # Compute cosine similarity with all keyword embeddings
        sims = cosine_similarity([embedding], keyword_embeds)[0]
        if sims.max() >= threshold:
            predicted_domains.append(domain)

    return predicted_domains

# Apply classification to all titles
df['predicted_domains'] = [classify_title(emb, domain_embeddings) for emb in tqdm(title_embeddings)]


100%|██████████| 9343/9343 [2:37:46<00:00,  1.01s/it]


In [ ]:
df.to_csv("/content/drive/MyDrive/casereports/titles_with_predicted_domains_1.csv", index=False)

In [ ]:
print("Titles -> Predicted Domains")

for i in range(10):
    title = df.loc[i, 'title']
    predicted = str(df.loc[i, 'predicted_domains'])
    print(f"{title} -> {predicted}")


Titles -> Predicted Domains
Bilateral Optic Neuroretinitis: Uncommon Complication of COVID-19. -> ['gene', 'taxonomy', 'brain_atlas', 'anatomy', 'molecule', 'disease', 'metadata', 'protein', 'phenotype', 'cognitive']
Bilateral Optic Neuropathy Associated with Acute Inhaled Marijuana Use: Case Report and Review of the Literature. -> ['gene', 'taxonomy', 'brain_atlas', 'anatomy', 'molecule', 'disease', 'metadata', 'protein', 'phenotype', 'cognitive']
Telemedicine in patients with obsessive-compulsive disorder after deep brain stimulation: a case series. -> ['gene', 'taxonomy', 'brain_atlas', 'anatomy', 'molecule', 'disease', 'metadata', 'protein', 'phenotype', 'cognitive']
Compulsive Repetitive Flexion With Breath-Holding in Sagging Brain Syndrome. -> ['gene', 'taxonomy', 'brain_atlas', 'anatomy', 'molecule', 'disease', 'metadata', 'protein', 'phenotype', 'cognitive']
Spontaneous Regression of Glioma-Mimicking Brainstem Lesion in a Child: A Case Report. -> ['gene', 'taxonomy', 'brain_atl

In [ ]:
# increasing threshold from 0.45 to 0.75
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Threshold for assigning a domain
THRESHOLD = 0.75

# Function to classify each title
def classify_title(embedding, domain_embeddings, threshold=THRESHOLD):
    predicted_domains = []

    for domain, keyword_embeds in domain_embeddings.items():
        # Compute cosine similarity with all keyword embeddings
        sims = cosine_similarity([embedding], keyword_embeds)[0]
        if sims.max() >= threshold:
            predicted_domains.append(domain)

    return predicted_domains

# Apply classification to all titles
df['predicted_domains'] = [classify_title(emb, domain_embeddings) for emb in tqdm(title_embeddings)]


100%|██████████| 9343/9343 [2:37:47<00:00,  1.01s/it]


In [ ]:
df.to_csv("/content/drive/MyDrive/casereports/titles_with_predicted_domains_2.csv", index=False)

In [ ]:
print("Titles -> Predicted Domains")

for i in range(50):
    title = df.loc[i, 'title']
    predicted = str(df.loc[i, 'predicted_domains'])
    print(f"{title} -> {predicted}")


Titles -> Predicted Domains
Bilateral Optic Neuroretinitis: Uncommon Complication of COVID-19. -> ['disease']
Bilateral Optic Neuropathy Associated with Acute Inhaled Marijuana Use: Case Report and Review of the Literature. -> []
Telemedicine in patients with obsessive-compulsive disorder after deep brain stimulation: a case series. -> []
Compulsive Repetitive Flexion With Breath-Holding in Sagging Brain Syndrome. -> []
Spontaneous Regression of Glioma-Mimicking Brainstem Lesion in a Child: A Case Report. -> []
Combined central and peripheral demyelination: a case report resembling encephalomyeloradiculoneuropathy. -> ['disease', 'metadata']
Case report: Headache as the sole neurological symptom in autoimmune glial fibrillary acidic protein (GFAP) astrocytopathy. -> []
Case report: Neuronal intranuclear inclusion disease initially mimicking reversible cerebral vasoconstriction syndrome: serial neuroimaging findings during an 11-year follow-up. -> []
Towards a neurocognitive profile in 

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

# Threshold for assigning a domain
THRESHOLD = 0.60

# Function to classify each title
def classify_title(embedding, domain_embeddings, threshold=THRESHOLD):
    predicted_domains = []

    for domain, keyword_embeds in domain_embeddings.items():
        # Compute cosine similarity with all keyword embeddings
        sims = cosine_similarity([embedding], keyword_embeds)[0]
        if sims.max() >= threshold:
            predicted_domains.append(domain)

    return predicted_domains

# Apply classification to all titles
df['predicted_domains'] = [classify_title(emb, domain_embeddings) for emb in tqdm(title_embeddings)]


100%|██████████| 9343/9343 [6:05:30<00:00,  2.35s/it]


In [7]:
df.to_csv("/content/drive/MyDrive/casereports/titles_with_predicted_domains_3.csv", index=False)

In [8]:
print("Titles -> Predicted Domains")

for i in range(50):
    title = df.loc[i, 'title']
    predicted = str(df.loc[i, 'predicted_domains'])
    print(f"{title} -> {predicted}")


Titles -> Predicted Domains
Bilateral Optic Neuroretinitis: Uncommon Complication of COVID-19. -> ['gene', 'taxonomy', 'brain_atlas', 'anatomy', 'molecule', 'disease', 'metadata', 'protein', 'phenotype']
Bilateral Optic Neuropathy Associated with Acute Inhaled Marijuana Use: Case Report and Review of the Literature. -> ['gene', 'taxonomy', 'brain_atlas', 'anatomy', 'molecule', 'disease', 'metadata', 'protein', 'phenotype']
Telemedicine in patients with obsessive-compulsive disorder after deep brain stimulation: a case series. -> ['gene', 'taxonomy', 'brain_atlas', 'anatomy', 'molecule', 'disease', 'metadata', 'protein', 'phenotype', 'cognitive']
Compulsive Repetitive Flexion With Breath-Holding in Sagging Brain Syndrome. -> ['gene', 'taxonomy', 'brain_atlas', 'anatomy', 'molecule', 'disease', 'metadata', 'protein', 'phenotype', 'cognitive']
Spontaneous Regression of Glioma-Mimicking Brainstem Lesion in a Child: A Case Report. -> ['taxonomy', 'anatomy', 'disease', 'metadata', 'protein']